In [38]:
import pandas as pd
df = pd.read_csv('Telco-Customer-Churn.csv')
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [39]:
df.isnull().sum() 


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [40]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') 
df.dropna(inplace=True)


In [41]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [42]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Drop customerID only if it exists
if 'customerID' in df.columns:
    df.drop('customerID', axis=1, inplace=True)

# Binary encoding for Churn
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Label encode all object columns and save encoders for ML model training
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Save all encoders
joblib.dump(label_encoders, 'label_encoders.pkl')


['label_encoders.pkl']

In [43]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# df['Churn'] = df['Churn'].map({1: 'Yes', 0: 'No'})

# sns.countplot(x='Churn', data=df)
# plt.title('Churn Distribution')


In [48]:
# contract_mapping = {
#     0: 'Month-to-month',
#     1: 'One year',
#     2: 'Two year'
# }
 
 
# df['ContractLabel'] = df['Contract'].map(contract_mapping)

# sns.countplot(x='ContractLabel', hue='Churn', data=df)
# plt.title('Churn by Contract Type')
# plt.xlabel('Contract Type')
# plt.ylabel('Number of Customers')
# plt.xticks(rotation=15)
# plt.show()


In [49]:
from sklearn.model_selection import train_test_split

X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Divide the Dataset into Train and test

In [50]:
!pip install xgboost


Defaulting to user installation because normal site-packages is not writeable


In [51]:
# Train Data on XGB classifier 
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:ContractLabel: object

In [ ]:
import joblib

# Save the trained XGBoost model
joblib.dump(model, 'xgb_churn_model.pkl')


['xgb_churn_model.pkl']

In [ ]:

import shap
import xgboost
import matplotlib.pyplot as plt

explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

In [52]:
shap.plots.beeswarm(shap_values)

NameError: name 'shap' is not defined

In [53]:
shap.plots.waterfall(shap_values[0])

NameError: name 'shap' is not defined